In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
import requests
from datetime import datetime, timedelta

In [6]:
!pip install ucimlrepo

In [7]:
from ucimlrepo import fetch_ucirepo

metro_interstate_traffic_volume = fetch_ucirepo(id=492)

X = metro_interstate_traffic_volume.data.features
y = metro_interstate_traffic_volume.data.targets

print(metro_interstate_traffic_volume.metadata)

print(metro_interstate_traffic_volume.variables)


{'uci_id': 492, 'name': 'Metro Interstate Traffic Volume', 'repository_url': 'https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume', 'data_url': 'https://archive.ics.uci.edu/static/public/492/data.csv', 'abstract': 'Hourly Minneapolis-St Paul, MN traffic volume for westbound I-94. Includes weather and holiday features from 2012-2018.', 'area': 'Other', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Sequential', 'Time-Series'], 'num_instances': 48204, 'num_features': 8, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['traffic_volume'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2019, 'last_updated': 'Fri Mar 15 2024', 'dataset_doi': '10.24432/C5X60B', 'creators': ['John Hogue'], 'intro_paper': None, 'additional_info': {'summary': 'Hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN. Hourly weath

- unos podatak pod imenom "Metro Interstate Traffic Volume" kodom sa same stranice na linku: https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume

In [28]:
X['date_time'] = pd.to_datetime(X['date_time'])
X['date'] = X['date_time'].dt.date

latitude = 44.9778
longitude = -93.2650

start_date = '2012-10-02'
end_date = '2018-09-30'

api_url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={latitude}&longitude={longitude}"
    f"&start_date={start_date}&end_date={end_date}"
    f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,"
    f"rain_sum,snowfall_sum,wind_speed_10m_max,wind_gusts_10m_max,"
    f"wind_direction_10m_dominant,shortwave_radiation_sum"
    f"&timezone=America%2FChicago"
)

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    if 'daily' in data:
        daily_data = data['daily']
        weather_df = pd.DataFrame(daily_data)
        weather_df['date'] = pd.to_datetime(weather_df['time']).dt.date
        weather_df.drop(columns=['time'], inplace=True)
    else:
        print("No data found")
else:
    print(f"Failed with stat. code: {response.status_code}")

- ovdje pozivamo open meteo historical weather API za nase podrucje Minneapolis, MN. za nas odredeni time frame od kuda su nam podaci iz prijašnjeg koraka
- u sljedećem koraku ćemo sve te podatke kombinirati u jedan csv koji ćemo i prikazati

In [29]:
combined_df = pd.merge(X, weather_df, on='date', how='left')

combined_df.to_csv('traffic_data.csv', index=False)

print("datasets have been combined")

Combined dataset saved as 'metro_traffic_with_weather.csv'.


In [30]:
df = pd.read_csv('traffic_data.csv')

print(df.head())

  holiday    temp  rain_1h  snow_1h  clouds_all weather_main  \
0     NaN  288.28      0.0      0.0          40       Clouds   
1     NaN  289.36      0.0      0.0          75       Clouds   
2     NaN  289.58      0.0      0.0          90       Clouds   
3     NaN  290.13      0.0      0.0          90       Clouds   
4     NaN  291.14      0.0      0.0          75       Clouds   

  weather_description            date_time        date  temperature_2m_max  \
0    scattered clouds  2012-10-02 09:00:00  2012-10-02                21.7   
1       broken clouds  2012-10-02 10:00:00  2012-10-02                21.7   
2     overcast clouds  2012-10-02 11:00:00  2012-10-02                21.7   
3     overcast clouds  2012-10-02 12:00:00  2012-10-02                21.7   
4       broken clouds  2012-10-02 13:00:00  2012-10-02                21.7   

   temperature_2m_min  precipitation_sum  rain_sum  snowfall_sum  \
0                 8.1                0.0       0.0           0.0   
1         